<a href="https://colab.research.google.com/github/ejrtks1020/Pytorch-basic/blob/main/pytorch-tutorials/pytorch_model_optimization_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##reference : https://tutorials.pytorch.kr/beginner/basics/optimization_tutorial.html

# 모델 매개변수 최적화
모델이 학습하는 과정
* 각 epoch에서 입력에 대해 출력을 계산(추측)
* 출력값과 실제 정답 사이 손실(Loss)을 계산
* 매개변수에 대한 Loss의 도함수(gradient)를 계산
* 경사하강법을 사용하여 이 파라미터들을 최적화(optimize)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



# 하이퍼 파라미터(Hyperparameter)
모델 최적화 과정을 제어할수 있는 조절 가능한 매개변수

서로다른 하이퍼파라미터값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있다.

하이퍼 파라미터 예)
* epoch - 데이터셋을 반복하는 횟수
* batch size - 매개변수가 갱신되기 전 신경망을 통해 전파되는 데이터샘플 수
* 학습률(learning rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습속도가 느려지고, 값이 클수록 발산하는등 예측할수 없는 동작이 발생할 수 있다.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# 최적화 단계 (Optimization Loop)
하이퍼 파라미터를 성절하고 최적화 단계를 통해 모델을 학습하고 최적화 할 수 있다. 최적화 단계의 각 반복(iteration)을 epoch이라고 부른다.

하나의 에폭은 두 부분으로 구성된다.
* 학습단계(train loop) - 학습용 데이터셋을 반복(순회, iterate)하고 최적의 매개변수로 수렴
* 검증/테스트 단계(validation/test loop) - 모델 성능이 개선되고 있는지 확인하기 위해 테스트 데이터셋을 반복(iterate)

# 손실함수(loss function)
* 학습용 데이터를 제공하면 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높다. 

* 손실함수는 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정하며, 학습중에 이값을 최소화하려고 한다. 

* 주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 loss를 계산한다

* 일반적인 손실함수에는 회귀문제(regression task)에 사용하는 nn.MSELoss(평균 제곱오차(MSE; Mean Square Error))나 분류(classification)에 사용하는 nn.NLLLOSS(음의 로그 우도(Negative Log Likelihood)), 그리고 nn.LogSoftmax와 nn.NLLLose를 합친 nn.CrossEntropyLoss 등이 있다.

* 모델의 출력 로짓(logit)을 nn.CrossEntropyLoss에 전달하여 로짓(logit)을 정규화 하고 예측 오류를 계산한다.

In [3]:
# 손실 함수를 초기화
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저(Optimizer)

최적화는 각 학습단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정.

옵티마이저 종류
* SGD(Stochastic Gradient Descent)
* ADAM
* RMSProp


In [4]:
# 학습하려는 모델의 매개변수와 학습률(learning rate) 하이퍼파라미터를 등록하여 옵티마이저 초기화
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# 최적화 구현

In [37]:
# 최적화 코드를 반복하여 수행하는 train_loop
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    # 예측(prediction)과 손실(loss) 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  
  # 한 iterate에서의 평균 loss
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg Loss: {test_loss:>8f} \n")

In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
  print(f"Epoch {t+1}\n--------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done")

Epoch 1
--------------------------
loss: 2.308007  [    0/60000]
loss: 2.295506  [ 6400/60000]
loss: 2.278783  [12800/60000]
loss: 2.274187  [19200/60000]
loss: 2.249230  [25600/60000]
loss: 2.223369  [32000/60000]
loss: 2.238587  [38400/60000]
loss: 2.200563  [44800/60000]
loss: 2.199158  [51200/60000]
loss: 2.171705  [57600/60000]
Test Error: 
 Accuracy: 47.4%, Avg loss: 2.165185 

Epoch 2
--------------------------
loss: 2.172406  [    0/60000]
loss: 2.162490  [ 6400/60000]
loss: 2.110343  [12800/60000]
loss: 2.132432  [19200/60000]
loss: 2.069512  [25600/60000]
loss: 2.011706  [32000/60000]
loss: 2.048134  [38400/60000]
loss: 1.965850  [44800/60000]
loss: 1.967065  [51200/60000]
loss: 1.907165  [57600/60000]
Test Error: 
 Accuracy: 57.2%, Avg loss: 1.901999 

Epoch 3
--------------------------
loss: 1.924632  [    0/60000]
loss: 1.899264  [ 6400/60000]
loss: 1.785703  [12800/60000]
loss: 1.837971  [19200/60000]
loss: 1.718721  [25600/60000]
loss: 1.659359  [32000/60000]
loss: 1.691